In [1]:
import dask.dataframe as dd
import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN

In [2]:

gdf = gpd.read_parquet('/data/uscuni-ulce/cadastre_buildings_standardised/buildings_lithuania.pq')


In [3]:
cents = gdf.centroid
gdf['x'], gdf['y'] = cents.x, cents.y
gdf['id'] = gdf.index.values
data = gdf[["x", "y", 'id']]

In [4]:
data[["x_100", "y_100"]] = np.around(data[["x", "y"]], decimals=-2)
grid = data[["id", "x_100", "y_100"]].groupby(["x_100", "y_100"]).count().reset_index()

/tmp/ipykernel_521548/4289293720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[["x_100", "y_100"]] = np.around(data[["x", "y"]], decimals=-2)
/tmp/ipykernel_521548/4289293720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[["x_100", "y_100"]] = np.around(data[["x", "y"]], decimals=-2)


In [5]:
dbscan = DBSCAN(400, n_jobs=-1).fit(grid[["x_100", "y_100"]], sample_weight=grid["id"])
grid["labels"] = dbscan.labels_

In [6]:
%%time
data = pd.merge(data, grid, "left", on=["x_100", "y_100"])

counts = data.labels.value_counts()
data["core"] = data.labels.isin(counts[counts > 10000].index.drop(-1))

cores = data[data.core]

grid["core"] = grid.labels.isin(counts[counts > 10000].index.drop(-1))
grid_cores = grid[grid.core]
grid_cores = gpd.GeoDataFrame(
    grid_cores["labels"],
    geometry=gpd.points_from_xy(grid_cores["x_100"], grid_cores["y_100"]),
    crs=3035,
)
grid_cores_dissolved = grid_cores.dissolve("labels")

grid_non_cores = grid[~grid.core]
grid_non_cores = gpd.GeoDataFrame(
    grid_non_cores["labels"],
    geometry=gpd.points_from_xy(grid_non_cores["x_100"], grid_non_cores["y_100"]),
    crs=3035,
)

grid_non_cores_clustered = grid_non_cores[grid_non_cores.labels != -1]
grid_non_cores_outliers = grid_non_cores[grid_non_cores.labels == -1]

grid_non_cores_clustered_dissolved = grid_non_cores_clustered.dissolve("labels")

CPU times: user 664 ms, sys: 38 ms, total: 702 ms
Wall time: 701 ms


In [7]:
%%time
nearest = grid_cores.sindex.nearest(
    grid_non_cores_clustered_dissolved.geometry, return_all=False
)

CPU times: user 361 ms, sys: 78 μs, total: 361 ms
Wall time: 360 ms


In [8]:
grid_non_cores_clustered_dissolved["nearest_core"] = grid_cores.labels.values[
    nearest[1]
]

In [9]:
nearest_outliers = grid_cores.sindex.nearest(
    grid_non_cores_outliers.geometry, return_all=False
)

In [10]:
grid_non_cores_outliers["nearest_core"] = grid_cores.labels.values[nearest_outliers[1]]

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [11]:
grid_non_cores = pd.concat(
    [
        grid_non_cores_clustered_dissolved.reset_index().explode(ignore_index=True),
        grid_non_cores_outliers,
    ],
    ignore_index=True,
)

In [12]:
grid_non_cores["x_100"] = grid_non_cores.geometry.x
grid_non_cores["y_100"] = grid_non_cores.geometry.y

data = pd.merge(
    data,
    grid_non_cores[["x_100", "y_100", "nearest_core"]],
    "left",
    on=["x_100", "y_100"],
)

data["region"] = data.labels
data.loc[~data.core, "region"] = data.loc[~data.core, "nearest_core"]

data = data.rename(
    columns={
        "id_x": "id",
        "id_y": "weight",
        "labels": "dbscan_cluster",
    }
)

In [13]:
additional_region_hulls = pd.concat(
    [
        grid_cores,
        grid_non_cores[["nearest_core", "geometry"]].rename(
            columns={"nearest_core": "labels"}
        ),
    ]
).dissolve("labels").convex_hull.to_frame("convex_hull")

In [14]:
data

,x,y,id,x_100,y_100,weight,dbscan_cluster,core,nearest_core,region
0,5.212683e+06,3.752221e+06,0,5212700.0,3752200.0,22,8003,False,8038.0,8038
1,5.192402e+06,3.547890e+06,1,5192400.0,3547900.0,1,7125,False,6310.0,6310
2,5.221373e+06,3.530085e+06,2,5221400.0,3530100.0,7,8795,False,8422.0,8422
3,5.215968e+06,3.543845e+06,3,5216000.0,3543800.0,24,8148,False,8422.0,8422
4,5.230448e+06,3.745754e+06,4,5230400.0,3745800.0,1,9249,False,8038.0,8038
...,...,...,...,...,...,...,...,...,...,...
2053124,5.196465e+06,3.539743e+06,2053124,5196500.0,3539700.0,11,7170,False,8422.0,8422
2053125,5.141297e+06,3.555753e+06,2053125,5141300.0,3555800.0,11,4295,False,6310.0,6310
2053126,5.201475e+06,3.748615e+06,2053126,5201500.0,3748600.0,7,7714,False,8038.0,8038
2053127,5.198955e+06,3.745027e+06,2053127,5199000.0,3745000.0,17,7432,False,8038.0,8038


In [15]:
regions_datadir = '/data/uscuni-ulce/'
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
    )

In [19]:
buffer = 139096 + 500

In [20]:
additional_region_hulls.index = additional_region_hulls.index + buffer
data['region'] = data['region'] + buffer

In [36]:
additional_region_hulls.to_parquet(regions_datadir + "regions/" + "lithuatia_cadastre_regions_hull.parquet")

In [37]:
region_hulls = pd.concat((region_hulls, additional_region_hulls))

In [41]:
region_hulls.to_parquet(regions_datadir + "regions/" + "cadastre_regions_hull.parquet")

In [46]:
additional_region_hulls.index

Index([139621, 139674, 139764, 140420, 141272, 141366, 142100, 143701, 145906,
       146285, 147112, 147634, 148018, 148085, 149997, 150044, 151676, 152081,
       152511],
      dtype='int64', name='labels')

In [29]:
data['region'].value_counts().index.sort_values()

Index([139621, 139674, 139764, 140420, 141272, 141366, 142100, 143701, 145906,
       146285, 147112, 147634, 148018, 148085, 149997, 150044, 151676, 152081,
       152511],
      dtype='int64', name='region')

### Assign buildings to regions

In [30]:
# 107134 - krakow
# 86876 = vienna
# 69336 = prague
# 4 - rhineruhr
#55765 - berlin
#bratislava - 99889
# hamburg - 16242
# freiburg - 5883

In [31]:
region_mapping = data

In [32]:
%%time
for region_id, group in region_mapping.groupby('region'):

    region_id = int(region_id)
    
    buildings = gdf.iloc[group.id]
    buildings['iid'] = buildings.index.values
    buildings.to_parquet(f'/data/uscuni-ulce/regions/buildings/buildings_{region_id}.pq')

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value

CPU times: user 3.43 s, sys: 616 ms, total: 4.05 s
Wall time: 4.03 s


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [6]:
region_name = 69333
regions_datadir = "/data/uscuni-ulce/"
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
    )
region_id, region_hull = region_hulls.loc[region_name].name, region_hulls.loc[region_name].convex_hull

In [8]:
# gpd.GeoSeries([region_hull], name='geometry', crs='epsg:3035').explore()